In [11]:
import findspark
findspark.init()

In [12]:
import pyspark
from pyspark.sql import SparkSession

In [14]:
spark=SparkSession.builder.appName('PySparkSqlApp').getOrCreate()

In [7]:
pdf=spark.read.csv('d:\\employees.csv',header=True,inferSchema=True)

In [8]:
pdf.printSchema()

root
 |-- First Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Last Login Time: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Bonus %: double (nullable = true)
 |-- Senior Management: boolean (nullable = true)
 |-- Team: string (nullable = true)



In [9]:
from pyspark.sql.functions import col

In [10]:
pdf1=pdf.select(col('First Name').alias('FirstName'),
                col('Gender').alias('Gen'),
                col('Start Date').alias('StartDate'),
                col('Last Login Time').alias('LastLogin'),
                col('Salary').alias('Sal'),
                col('Bonus %').alias('Bonus'),
                col('Senior Management').alias('Sm'),
                col('Team').alias('team'))
                
pdf1.show()               

+---------+------+----------+---------+------+------+-----+--------------------+
|FirstName|   Gen| StartDate|LastLogin|   Sal| Bonus|   Sm|                team|
+---------+------+----------+---------+------+------+-----+--------------------+
|  Douglas|  Male|08-06-1993| 12:42 PM| 97308| 6.945| true|           Marketing|
|   Thomas|  Male| 3/31/1996|  6:53 AM| 61933|  4.17| true|                null|
|    Maria|Female| 4/23/1993| 11:17 AM|  null|11.858|false|             Finance|
|    Jerry|  Male|03-04-2005|  1:00 PM|138705|  null| true|             Finance|
|    Larry|  Male| 1/24/1998|  4:47 PM|  null| 1.389| true|     Client Services|
|   Dennis|  Male| 4/18/1987|  1:35 AM|115163|10.125|false|               Legal|
|     Ruby|Female|      null|     null| 65476|10.012| true|             Product|
|     null|Female| 7/20/2015| 10:43 AM|  null|  null| null|             Finance|
|   Angela|Female|      null|  6:29 AM| 95570|  null| true|         Engineering|
|  Frances|Female|      null

In [11]:
pdf1.createOrReplaceTempView('tempview')

In [16]:
result=spark.sql('select team,sum(Sal) as Total_sal from tempview GROUP BY team').toPandas()

In [17]:
result

,team,Total_sal
0,Sales,8664303
1,Engineering,8672766
2,None,3902815
3,Business Development,9278498
4,Finance,9229891
5,Client Services,9250785
6,Distribution,7965042
7,Legal,7858718
8,Marketing,8862688
9,Product,8423223


In [18]:
from pyspark.ml.feature import SQLTransformer 

In [22]:
sqltrans=SQLTransformer(statement='select FirstName,team,Sal,Bonus from __THIS__')
sqltrans.transform(pdf1).show(5)

+---------+---------------+------+------+
|FirstName|           team|   Sal| Bonus|
+---------+---------------+------+------+
|  Douglas|      Marketing| 97308| 6.945|
|   Thomas|           null| 61933|  4.17|
|    Maria|        Finance|  null|11.858|
|    Jerry|        Finance|138705|  null|
|    Larry|Client Services|  null| 1.389|
+---------+---------------+------+------+
only showing top 5 rows



In [23]:
pd=spark.read.csv('d:\\item_details.csv',header=True,inferSchema=True)

In [24]:
pd.printSchema()

root
 |-- item id: integer (nullable = true)
 |-- item name: string (nullable = true)
 |-- item cost: double (nullable = true)
 |-- supplier: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- qty: integer (nullable = true)



In [27]:
pd1=pd.select(col('item name').alias('iname'),
                col('item cost').alias('icost'))
                
                
pd1.show()               

+------+-----+
| iname|icost|
+------+-----+
| chalk|65.76|
|Pencil| null|
|   Pen| 43.0|
|Duster| 54.0|
| Scale| 10.0|
| chalk|65.76|
+------+-----+



In [29]:
sqltrans=SQLTransformer(statement='select max(icost),min(icost),sum(icost) from __THIS__')
sqltrans.transform(pd1).show()

+----------+----------+------------------+
|max(icost)|min(icost)|        sum(icost)|
+----------+----------+------------------+
|     65.76|      10.0|238.51999999999998|
+----------+----------+------------------+



In [31]:
#expr()
from pyspark.sql.functions import expr

In [38]:
pdf1.withColumn('Increment',expr('Sal+2000')).show()

+---------+------+----------+---------+------+------+-----+--------------------+---------+
|FirstName|   Gen| StartDate|LastLogin|   Sal| Bonus|   Sm|                team|Increment|
+---------+------+----------+---------+------+------+-----+--------------------+---------+
|  Douglas|  Male|08-06-1993| 12:42 PM| 97308| 6.945| true|           Marketing|    99308|
|   Thomas|  Male| 3/31/1996|  6:53 AM| 61933|  4.17| true|                null|    63933|
|    Maria|Female| 4/23/1993| 11:17 AM|  null|11.858|false|             Finance|     null|
|    Jerry|  Male|03-04-2005|  1:00 PM|138705|  null| true|             Finance|   140705|
|    Larry|  Male| 1/24/1998|  4:47 PM|  null| 1.389| true|     Client Services|     null|
|   Dennis|  Male| 4/18/1987|  1:35 AM|115163|10.125|false|               Legal|   117163|
|     Ruby|Female|      null|     null| 65476|10.012| true|             Product|    67476|
|     null|Female| 7/20/2015| 10:43 AM|  null|  null| null|             Finance|     null|

In [45]:
pdf1.selectExpr('FirstName','Sal','Sal+2000').show()

+---------+------+------------+
|FirstName|   Sal|(Sal + 2000)|
+---------+------+------------+
|  Douglas| 97308|       99308|
|   Thomas| 61933|       63933|
|    Maria|  null|        null|
|    Jerry|138705|      140705|
|    Larry|  null|        null|
|   Dennis|115163|      117163|
|     Ruby| 65476|       67476|
|     null|  null|        null|
|   Angela| 95570|       97570|
|  Frances|139852|      141852|
|   Louise| 63241|       65241|
|    Julie|102508|      104508|
|  Brandon|112807|      114807|
|     Gary|109831|      111831|
| Kimberly| 41426|       43426|
|  Lillian| 59414|       61414|
|   Jeremy| 90370|       92370|
|    Shawn|111737|      113737|
|    Diana|132940|      134940|
|    Donna| 81014|       83014|
+---------+------+------------+
only showing top 20 rows



In [46]:
pd1.withColumn('Increment',expr('icost+2000')).show()

+------+-----+---------+
| iname|icost|Increment|
+------+-----+---------+
| chalk|65.76|  2065.76|
|Pencil| null|     null|
|   Pen| 43.0|   2043.0|
|Duster| 54.0|   2054.0|
| Scale| 10.0|   2010.0|
| chalk|65.76|  2065.76|
+------+-----+---------+



In [48]:
pd1.selectExpr('iname','icost','icost+200').show()

+------+-----+-------------+
| iname|icost|(icost + 200)|
+------+-----+-------------+
| chalk|65.76|       265.76|
|Pencil| null|         null|
|   Pen| 43.0|        243.0|
|Duster| 54.0|        254.0|
| Scale| 10.0|        210.0|
| chalk|65.76|       265.76|
+------+-----+-------------+



In [51]:
pdf1.withColumn('Gen',expr('CASE WHEN Gen="Male" Then "M"' + 'WHEN Gen="Female" Then "F" END' )).show()

+---------+---+----------+---------+------+------+-----+--------------------+
|FirstName|Gen| StartDate|LastLogin|   Sal| Bonus|   Sm|                team|
+---------+---+----------+---------+------+------+-----+--------------------+
|  Douglas|  M|08-06-1993| 12:42 PM| 97308| 6.945| true|           Marketing|
|   Thomas|  M| 3/31/1996|  6:53 AM| 61933|  4.17| true|                null|
|    Maria|  F| 4/23/1993| 11:17 AM|  null|11.858|false|             Finance|
|    Jerry|  M|03-04-2005|  1:00 PM|138705|  null| true|             Finance|
|    Larry|  M| 1/24/1998|  4:47 PM|  null| 1.389| true|     Client Services|
|   Dennis|  M| 4/18/1987|  1:35 AM|115163|10.125|false|               Legal|
|     Ruby|  F|      null|     null| 65476|10.012| true|             Product|
|     null|  F| 7/20/2015| 10:43 AM|  null|  null| null|             Finance|
|   Angela|  F|      null|  6:29 AM| 95570|  null| true|         Engineering|
|  Frances|  F|      null|  6:51 AM|139852| 7.524| true|Business

In [61]:
pd1.withColumn('Grade',expr('CASE WHEN icost>=100 Then "A"' + 'WHEN icost>=50 Then "B" Else "C" END' )).show()

+------+-----+-----+
| iname|icost|Grade|
+------+-----+-----+
| chalk|65.76|    B|
|Pencil| null|    C|
|   Pen| 43.0|    C|
|Duster| 54.0|    B|
| Scale| 10.0|    C|
| chalk|65.76|    B|
+------+-----+-----+



In [53]:
pd1.show()

+------+-----+
| iname|icost|
+------+-----+
| chalk|65.76|
|Pencil| null|
|   Pen| 43.0|
|Duster| 54.0|
| Scale| 10.0|
| chalk|65.76|
+------+-----+



In [63]:
pdf1.withColumn('Bonus',expr('round(Bonus,2)')).show()

+---------+------+----------+---------+------+-----+-----+--------------------+
|FirstName|   Gen| StartDate|LastLogin|   Sal|Bonus|   Sm|                team|
+---------+------+----------+---------+------+-----+-----+--------------------+
|  Douglas|  Male|08-06-1993| 12:42 PM| 97308| 6.95| true|           Marketing|
|   Thomas|  Male| 3/31/1996|  6:53 AM| 61933| 4.17| true|                null|
|    Maria|Female| 4/23/1993| 11:17 AM|  null|11.86|false|             Finance|
|    Jerry|  Male|03-04-2005|  1:00 PM|138705| null| true|             Finance|
|    Larry|  Male| 1/24/1998|  4:47 PM|  null| 1.39| true|     Client Services|
|   Dennis|  Male| 4/18/1987|  1:35 AM|115163|10.13|false|               Legal|
|     Ruby|Female|      null|     null| 65476|10.01| true|             Product|
|     null|Female| 7/20/2015| 10:43 AM|  null| null| null|             Finance|
|   Angela|Female|      null|  6:29 AM| 95570| null| true|         Engineering|
|  Frances|Female|      null|  6:51 AM|1

In [70]:
pdf1.select('FirstName','Sal','Bonus' ,expr('round(Bonus,2) as roundup ')).show()

+---------+------+------+-------+
|FirstName|   Sal| Bonus|roundup|
+---------+------+------+-------+
|  Douglas| 97308| 6.945|   6.95|
|   Thomas| 61933|  4.17|   4.17|
|    Maria|  null|11.858|  11.86|
|    Jerry|138705|  null|   null|
|    Larry|  null| 1.389|   1.39|
|   Dennis|115163|10.125|  10.13|
|     Ruby| 65476|10.012|  10.01|
|     null|  null|  null|   null|
|   Angela| 95570|  null|   null|
|  Frances|139852| 7.524|   7.52|
|   Louise| 63241|15.132|  15.13|
|    Julie|102508|12.637|  12.64|
|  Brandon|112807|17.492|  17.49|
|     Gary|109831| 5.831|   5.83|
| Kimberly| 41426|14.543|  14.54|
|  Lillian| 59414| 1.256|   1.26|
|   Jeremy| 90370| 7.369|   7.37|
|    Shawn|111737| 6.414|   6.41|
|    Diana|132940|19.082|  19.08|
|    Donna| 81014| 1.894|   1.89|
+---------+------+------+-------+
only showing top 20 rows



In [76]:
pdf1.selectExpr('FirstName','Bonus' ,'round(Bonus,2) as roundup ').show(4)

+---------+------+-------+
|FirstName| Bonus|roundup|
+---------+------+-------+
|  Douglas| 6.945|   6.95|
|   Thomas|  4.17|   4.17|
|    Maria|11.858|  11.86|
|    Jerry|  null|   null|
+---------+------+-------+
only showing top 4 rows



In [77]:
pdf1.selectExpr('FirstName','round(Bonus,2)').filter('FirstName="Jerry"').show()

+---------+---------------+
|FirstName|round(Bonus, 2)|
+---------+---------------+
|    Jerry|           null|
|    Jerry|           19.1|
|    Jerry|           9.18|
|    Jerry|          18.85|
|    Jerry|          11.39|
|    Jerry|          18.86|
+---------+---------------+



In [113]:
pdf1.createOrReplaceTempView('tempview')

In [141]:
spark.sql('select team,sum(Sal) from tempview where length(team)==7 GROUP BY team').show()

+-------+--------+
|   team|sum(Sal)|
+-------+--------+
|Finance| 9229891|
|Product| 8423223|
+-------+--------+



In [110]:
pdf1.selectExpr("team","Sal").filter('length(team)=="7"').groupBy('team').sum('Sal').show()

+-------+--------+
|   team|sum(Sal)|
+-------+--------+
|Finance| 9229891|
|Product| 8423223|
+-------+--------+



In [108]:
pdf1.printSchema()

root
 |-- FirstName: string (nullable = true)
 |-- Gen: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- LastLogin: string (nullable = true)
 |-- Sal: integer (nullable = true)
 |-- Bonus: double (nullable = true)
 |-- Sm: boolean (nullable = true)
 |-- team: string (nullable = true)



In [142]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [16]:
import mysql.connector as c
import pandas as pd

In [17]:
conn=c.Connect(host='localhost',port=3306,user='root',password='mypass',database='travelling')
cursor=conn.cursor()


In [9]:
p1=pd.read_sql('select busno,busName from bus',con=conn)
print(p1)
print(type(p1))
conn.close()

   busno          busName
0   1234    PanjiToMumbai
1  12344             PTOM
2  12347             PTOM
3  14567      bgmtopunjab
4  23456      MumbaiToGoa
5  34566  PunaToBangalore
6  56778   PanjiToBelgaum
7  67890    BelgaumToPuna
<class 'pandas.core.frame.DataFrame'>


In [4]:
pip install SQLAlchemy

In [20]:
spark.createDataFrame(pd.read_sql('select busno,busName from bus',con=conn)).show()

D:\New folder\Python\envs\envr1\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


+-----+---------------+
|busno|        busName|
+-----+---------------+
| 1234|  PanjiToMumbai|
|12344|           PTOM|
|12347|           PTOM|
|14567|    bgmtopunjab|
|23456|    MumbaiToGoa|
|34566|PunaToBangalore|
|56778| PanjiToBelgaum|
|67890|  BelgaumToPuna|
+-----+---------------+

